# 1. Introduction: Advanced programming for economic statistics

Welcome to the SSRIC Instructional Modules for the project, "Teaching Statistics and Economic Data Analysis in Python with Jupyter Notebooks", by Daniel MacDonald, Associate Professor and Chair, Economics Department, CSU San Bernardino. These were written in Summer 2023.

Most of the modules draw extensively on Kevin Sheppard's e-book, *Introduction to Python for Econometrics, Statistics, and Data Analysis*, available here: https://bashtage.github.io/kevinsheppard.com/files/teaching/python/notes/python_introduction_2021.pdf. 

Rather than begin instruction in Python through the core tools of computer programming (such as conditions, loops, and functions), Sheppard begins with Python's major "containers", or data structures. Through practice, I have learned that this is an effective method for teaching Python to economics majors.

The learning objectives of this set of Instructional Modules are as follows. By the end of these modules, students will be able to...

1. Create data structures in Python based on economic data
1. Summarize the statistical properties of economic data (median, mean, max, min, correlation) using Python
1. Create and manage economic data: create new columns and rows, merge and append, and import data from .csv and .xlsx files into Python
1. Visualize economic data using line and scatterplots

The objectives/content of Module 8, which is the final module, are as follows. Students will be able to...

1. Think about conditional means as "if-then" statements
1. Use conditional means to calculate summary statistics
1. Learn how Pandas deals with null or `NaN` values

In [ ]:
import pandas as pd

## 1.1 Overview

For this module, we will use Current Population Survey data from 2021, extracted from IPUMS. I created an extract of the basic monthly samples from January 2021-December 2022 from IPUMS. I then reduced the number of variables to the ones needed for this module, and created a California-only subsample, which is used here.

A common problem in economic statistics is the calculation of *conditional* statistics: what is the unemployment rate among men or women specifically? What is the unemployment rate among African Americans? 

When we ask these questions, we are implicitly forming an *if-then* statement: 

1. *If* a member of the labor force is female, *then* what is the probability she is unemployed? (Unemployment rate.) 
1. *If* a member of the population is Hispanic, *then* what is the probability that they are in the labor force? (Labor force participation rate.)

Like all programming languages, Python is equipped to address if-then statements and provides many ways to perform analysis on the basis of conditional statements. Pandas in particular allows us to construct conditional statistics using some of the techniques we have already learned.

In [ ]:
df=pd.read_csv('module 8 data.csv')

In [ ]:
print(df.columns)

In [ ]:
print(df[['serial', 'sex', 'race']].head(10))

We are printing just a few rows and columns from this DataFrame, just to get a sense of what the data look like. `serial` refers to the household ID; there are 2 people in the first household, 5 person in the second, at least 3 in the third household, and so on. `sex` takes on a value of 1 or 2 and indicates whether a person is male or female respectively. `race` refers to a person's race; here, we only see three values: 100, 651, and 803; from a codebook we can learn that race=100 means the person is White, race=651 means the person is Asian, and 803 means the person identifies as 2 races: White and Asian.

# 2. Calculating basic statistics using dummy variables

We begin with two non-economic examples: suppose we want to calculate the percent male, and the percent African-American, in this dataset.

There is no automatic way to do this using Python or Pandas. Checking the values of `sex` or `race` in the CPS will give you raw counts, and referring to a codebook would tell you what the codes refer to, but nothing will tell you percentages. 

To solve this problem, suppose you have a dataset with 10 people; 4 of them are male and 6 of them are female. You could create a new column titled "male", assign a value of 1 to each male, and a value of 0 to each female. Taking the average of this column would give you the percent male: $\frac{1+1+1+1+0+0+0+0+0+0}{10}=0.4=40\%$

This is the idea behind the **dummy variable**. Dummy variables assign a value of 1 *if* an observation assumes some characteristic, and zero otherwise. Notice the language of *if*: this is essentially a programming question, as we will see in a minute:

In [ ]:
df['sex'].value_counts()

In [ ]:
df.loc[df['sex']==1, 'male']=1
df.loc[df['sex']==2, 'male']=0

In [ ]:
df['male'].mean()

## 2.1 Comment on above code

The code above creates a new column in our DataFrame called `male`. Each row of that column is calculated as follows:

1. *If* the row in that column has a value of `sex` equal to 1 (meaning, according to the CPS codebook, they are a male), then it assigns the value of `male` equal to 1
1. *If* the row in that column has a value of `sex` equal to 2 (meaning they are a female), then it assigns the value of `male` equal to 0

Calculating the average of this newly-created column gives us the percentage male in our DataFrame.

In [ ]:
print(df['race'].value_counts())

In [ ]:
df.loc[df['race']==200, 'black_only']=1
df.loc[df['race']!=200, 'black_only']=0

In [ ]:
print(df['black_only'].mean())

## 2.2 Comment on the above code

According to the CPS codebook: https://cps.ipums.org/cps-action/variables/race#codes_section.

An observation with `race` code equalling 200 means that the person is Black. We thus create a new dummy variable, "black", which only equals 1 if the person's race code is 200. Otherwise, the value is zero. Calculating the average of this column gives us the percent of the sample that is Black in the California subsample of the CPS.

# 3. Labor force statistics: labor force participation and unemployment rate

Labor force statistics are slightly more complicated. As you see in the first block of code below, `labforce` can be one of three values: 0, 1, or 2:

In [ ]:
print(df['labforce'].value_counts())

The value of 2 indicates "in labor force" and the value of 1 indicates "not in labor force". So, what does the value of 0 refer to?

The value of 0 is "not in universe". As you learn in Principles of Macroeconomics, only people 16 or older can even be considered as in the labor force. If you're older than 16 but disabled, or retired, for example, then you are not in the labor force. But, if you are a 10 year old child, you can't work, and so you're not in the labor force. 

In [ ]:
df.loc[df['labforce']==2, 'ilf']=1 
df.loc[df['labforce']==1, 'ilf']=0

What do the observations with `labforce=0` look like? As we see below, **they take the value in Pandas of `NaN`**, which means "null" or "not a number". It's similar to an Excel cell that is blank - it doesn't even have a number registered in it.

These `NaN` observations will not be considered when you calculate averages. Python will pass over them so that they are not considered.

This is a very important point to realize when doing economic statistics: if some values are null, they will be ignored in Python (unless you tell it not to). Let's see what the labor force participation rate is: 

In [ ]:
print(df['ilf'][df['labforce']==0]) # We want to view entries for 'ilf', but only when 'labforce'==0

In [ ]:
print(df['ilf'].mean())

## 3.1 Comment on the above

You see that the value of `ilf`, when `labforce=0` (meaning, the person is "not in the universe" for consideration of being in the labor force), is `NaN`. So when we calculate the mean of `ilf`, **these observations will not be considered**. 

Indeed, the labor force participation rate that we obtained (keeping in mind this is 2021 and California only) is very close to official statistics of 61.0: https://fred.stlouisfed.org/series/LBSSA06#0

## 3.2 Calculating unemployment rate

The unemployment rate is calculated similarly. Recall that the unemployment rate (expressed as a percent) is:

$\frac{\text{unemployed}}{\text{labor force}}x100$

So, people not in the labor force are not counted as unemployed. You must be out of work and actively searching for work to be counted as unemployed.

Further, the labor force is $\text{employed}+\text{unemployed}$. Thus, the code below will correctly calculate the unemployment rate in California in 2021, based on the fact that the two "unemployed" codes in `empstat` are 21 and 22, while the two "employed" codes in `empstat` are 10 and 12: https://cps.ipums.org/cps-action/variables/empstat#codes_section

In [ ]:
unemployed_codes=[21, 22] 
employed_codes=[10, 12]
df.loc[df['empstat'].isin(unemployed_codes), 'unemployed']=1
df.loc[df['empstat'].isin(employed_codes), 'unemployed']=0

In [ ]:
urate=100*df['unemployed'].mean()
print(urate,'%')

The official statistics for California have the unemployment rate at 7.3% for 2021, so what we found is very close: https://fred.stlouisfed.org/series/CAUR#0.

# 4. Try it yourself

Try it yourself. For this final module, **first calculate the percent of the population that is Asian in California**, using the fact that race code = 651 is for observations that identify as asian.

**Second, calculate the employment rate in California**. Note that this is just a slight modification of the code to calculate the unemployment rate. 

Write your code for both tasks below:

In [ ]:
# Try it yourself: write your code below:

